# Data Cleaning

## Data Sources

[crashes](https://data.cityofchicago.org/Transportation/Traffic-Crashes-Crashes/85ca-t3if)

[vehicles](https://data.cityofchicago.org/Transportation/Traffic-Crashes-Vehicles/68nd-jvt3)

[people](https://data.cityofchicago.org/Transportation/Traffic-Crashes-People/u6pd-qa9d)

## Libraries

In [111]:
import pandas as pd
import scipy.sparse as sp
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer


## Importing The Data

In [112]:
d_crashes = pd.read_csv('data/Traffic_Crashes_-_Crashes.csv')
d_vehicles = pd.read_csv('data/Traffic_Crashes_-_Vehicles.csv')

c:\Anaconda3\envs\learn-env\lib\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (19,21,40,41,42,44,48,49,50,53,55,58,59,61,71) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


### Look at the dimensions of the matrices

In [113]:
d_crashes.shape

(732329, 49)

In [114]:
d_crashes.head()

,CRASH_RECORD_ID,RD_NO,CRASH_DATE_EST_I,CRASH_DATE,POSTED_SPEED_LIMIT,TRAFFIC_CONTROL_DEVICE,DEVICE_CONDITION,WEATHER_CONDITION,LIGHTING_CONDITION,FIRST_CRASH_TYPE,...,INJURIES_NON_INCAPACITATING,INJURIES_REPORTED_NOT_EVIDENT,INJURIES_NO_INDICATION,INJURIES_UNKNOWN,CRASH_HOUR,CRASH_DAY_OF_WEEK,CRASH_MONTH,LATITUDE,LONGITUDE,LOCATION
0,79c7a2ce89f446262efd86df3d72d18b04ba487024b7c4...,JC199149,NaN,03/25/2019 02:43:00 PM,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,TURNING,...,0.0,1.0,2.0,0.0,14,2,3,41.884547,-87.641201,POINT (-87.64120093714 41.884547224337)
1,792b539deaaad65ee5b4a9691d927a34d298eb33d42af0...,JB422857,NaN,09/05/2018 08:40:00 AM,30,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,ANGLE,...,0.0,0.0,2.0,0.0,8,4,9,41.968562,-87.740659,POINT (-87.740659314632 41.968562453871)
2,0115ade9a755e835255508463f7e9c4a9a0b47e9304238...,JF318029,NaN,07/15/2022 12:45:00 AM,30,UNKNOWN,UNKNOWN,CLEAR,"DARKNESS, LIGHTED ROAD",ANGLE,...,0.0,0.0,2.0,0.0,0,6,7,41.886336,-87.716203,POINT (-87.716203130599 41.886336409761)
3,05b1982cdba5d8a00e7e76ad1ecdab0e598429f78481d2...,JF378711,NaN,08/29/2022 11:30:00 AM,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,REAR END,...,0.0,0.0,3.0,0.0,11,2,8,41.749348,-87.721097,POINT (-87.721096727406 41.749348170421)
4,017040c61958d2fa977c956b2bd2d6759ef7754496dc96...,JF324552,NaN,07/15/2022 06:50:00 PM,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,REAR END,...,0.0,0.0,2.0,0.0,18,6,7,41.925111,-87.667997,POINT (-87.667997321599 41.925110815832)


In [115]:
d_vehicles.shape

(1491796, 72)

In [116]:
d_vehicles.head()

,CRASH_UNIT_ID,CRASH_RECORD_ID,RD_NO,CRASH_DATE,UNIT_NO,UNIT_TYPE,NUM_PASSENGERS,VEHICLE_ID,CMRC_VEH_I,MAKE,...,TRAILER1_LENGTH,TRAILER2_LENGTH,TOTAL_VEHICLE_LENGTH,AXLE_CNT,VEHICLE_CONFIG,CARGO_BODY_TYPE,LOAD_TYPE,HAZMAT_OUT_OF_SERVICE_I,MCS_OUT_OF_SERVICE_I,HAZMAT_CLASS
0,1554880,91a5d08b2b701f2d37cbb52ecdbeb09579bc7f2ebc60b3...,JG223284,04/14/2023 02:05:00 PM,1,DRIVER,NaN,1478881.0,NaN,FORD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,749947,81dc0de2ed92aa62baccab641fa377be7feb1cc47e6554...,JC451435,09/28/2019 03:30:00 AM,1,DRIVER,NaN,834816.0,NaN,HONDA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,749949,81dc0de2ed92aa62baccab641fa377be7feb1cc47e6554...,JC451435,09/28/2019 03:30:00 AM,2,PARKED,NaN,834819.0,NaN,TOYOTA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,749950,81dc0de2ed92aa62baccab641fa377be7feb1cc47e6554...,JC451435,09/28/2019 03:30:00 AM,3,PARKED,NaN,834817.0,NaN,GENERAL MOTORS CORPORATION (GMC),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1554881,91a5d08b2b701f2d37cbb52ecdbeb09579bc7f2ebc60b3...,JG223284,04/14/2023 02:05:00 PM,2,DRIVER,1.0,1478892.0,NaN,ISUZU,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Merging The Frames

In [117]:
# Merge d_crashes and d_vehicles on CRASH_RECORD_ID
df = pd.merge(d_crashes, d_vehicles, on='CRASH_RECORD_ID')

In [118]:
df.shape

(1491794, 120)

In [119]:
df.head()

,CRASH_RECORD_ID,RD_NO_x,CRASH_DATE_EST_I,CRASH_DATE_x,POSTED_SPEED_LIMIT,TRAFFIC_CONTROL_DEVICE,DEVICE_CONDITION,WEATHER_CONDITION,LIGHTING_CONDITION,FIRST_CRASH_TYPE,...,TRAILER1_LENGTH,TRAILER2_LENGTH,TOTAL_VEHICLE_LENGTH,AXLE_CNT,VEHICLE_CONFIG,CARGO_BODY_TYPE,LOAD_TYPE,HAZMAT_OUT_OF_SERVICE_I,MCS_OUT_OF_SERVICE_I,HAZMAT_CLASS
0,79c7a2ce89f446262efd86df3d72d18b04ba487024b7c4...,JC199149,NaN,03/25/2019 02:43:00 PM,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,TURNING,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,79c7a2ce89f446262efd86df3d72d18b04ba487024b7c4...,JC199149,NaN,03/25/2019 02:43:00 PM,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,TURNING,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,792b539deaaad65ee5b4a9691d927a34d298eb33d42af0...,JB422857,NaN,09/05/2018 08:40:00 AM,30,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,ANGLE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,792b539deaaad65ee5b4a9691d927a34d298eb33d42af0...,JB422857,NaN,09/05/2018 08:40:00 AM,30,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,ANGLE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0115ade9a755e835255508463f7e9c4a9a0b47e9304238...,JF318029,NaN,07/15/2022 12:45:00 AM,30,UNKNOWN,UNKNOWN,CLEAR,"DARKNESS, LIGHTED ROAD",ANGLE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Check dimensions

In [120]:
print(d_crashes.shape[1] + d_vehicles.shape[1])

121


## Initial Drop

In [121]:
'''
drom from df 
CMV_ID
USDOT_NO
CCMC_NO
ILCC_NO
COMMERCIAL_SRC
GVWR
CARRIER_NAME
CARRIER_STATE
CARRIER_CITY
HAZMAT_PLACARDS_I
HAZMAT_NAME
UN_NO
HAZMAT_PRESENT_I
HAZMAT_REPORT_I
HAZMAT_REPORT_NO
MCS_REPORT_I
MCS_REPORT_NO
HAZMAT_VIO_CAUSE_CRASH_I
MCS_VIO_CAUSE_CRASH_I
IDOT_PERMIT_NO
WIDE_LOAD_I
TRAILER1_WIDTH
TRAILER2_WIDTH
TRAILER1_LENGTH
TRAILER2_LENGTH
TOTAL_VEHICLE_LENGTH
AXLE_CNT
VEHICLE_CONFIG
CARGO_BODY_TYPE
LOAD_TYPE
HAZMAT_OUT_OF_SERVICE_I
MCS_OUT_OF_SERVICE_I
HAZMAT_CLASS
PHOTOS_TAKEN_I
STATEMENTS_TAKEN_I
DOORING_I
WORK_ZONE_I
WORK_ZONE_TYPE
WORKERS_PRESENT_I
TOWED_I
FIRE_I
TOWED_BY
TOWED_TO
AREA_00_I
AREA_01_I
AREA_02_I
AREA_03_I
AREA_04_I
AREA_05_I
AREA_06_I
AREA_07_I
AREA_08_I
AREA_09_I
AREA_10_I
AREA_11_I
AREA_12_I
AREA_99_I
'''

df.drop(columns=['CMV_ID',
                 'USDOT_NO',
                 'CCMC_NO',
                 'ILCC_NO',
                 'COMMERCIAL_SRC',
                 'GVWR',
                 'CARRIER_NAME',
                 'CARRIER_STATE',
                 'CARRIER_CITY',
                 'HAZMAT_PLACARDS_I',
                 'HAZMAT_NAME',
                 'UN_NO',
                 'HAZMAT_PRESENT_I',
                 'HAZMAT_REPORT_I',
                 'HAZMAT_REPORT_NO',
                 'MCS_REPORT_I',
                 'MCS_REPORT_NO',
                 'HAZMAT_VIO_CAUSE_CRASH_I',
                 'MCS_VIO_CAUSE_CRASH_I',
                 'IDOT_PERMIT_NO',
                 'WIDE_LOAD_I',
                 'TRAILER1_WIDTH',
                 'TRAILER2_WIDTH',
                 'TRAILER1_LENGTH',
                 'TRAILER2_LENGTH',
                 'TOTAL_VEHICLE_LENGTH',
                 'AXLE_CNT',
                 'VEHICLE_CONFIG',
                 'CARGO_BODY_TYPE',
                 'LOAD_TYPE',
                 'HAZMAT_OUT_OF_SERVICE_I',
                 'MCS_OUT_OF_SERVICE_I',
                 'HAZMAT_CLASS',
                 
                 'PHOTOS_TAKEN_I',
                 'STATEMENTS_TAKEN_I',
                 'DOORING_I',
                 'WORK_ZONE_I',
                 'WORK_ZONE_TYPE',
                 'WORKERS_PRESENT_I',
                 
                 'TOWED_I',
                 'FIRE_I',
                 
                 'TOWED_BY',
                 'TOWED_TO',
                 'AREA_00_I',
                 'AREA_01_I',
                 'AREA_02_I',
                 'AREA_03_I',
                 'AREA_04_I',
                 'AREA_05_I',
                 'AREA_06_I',
                 'AREA_07_I',
                 'AREA_08_I',
                 'AREA_09_I',
                 'AREA_10_I',
                 'AREA_11_I',
                 'AREA_12_I',
                 'AREA_99_I',
                 
                 'BEAT_OF_OCCURRENCE',
                 'VEHICLE_ID',
                 'LANE_CNT',
                 ], inplace=True)

In [122]:
df.shape

(1491794, 60)

In [123]:
# remove all rows where PRIM_CONTRIBUTORY_CAUSE is UNABLE TO DETERMINE
df = df[df['PRIM_CONTRIBUTORY_CAUSE'] != 'UNABLE TO DETERMINE']



In [124]:
# check
df['PRIM_CONTRIBUTORY_CAUSE'].value_counts()

FAILING TO YIELD RIGHT-OF-WAY                                                       164915
FOLLOWING TOO CLOSELY                                                               151936
NOT APPLICABLE                                                                       75316
IMPROPER OVERTAKING/PASSING                                                          73063
FAILING TO REDUCE SPEED TO AVOID CRASH                                               66695
IMPROPER BACKING                                                                     58478
IMPROPER LANE USAGE                                                                  54856
IMPROPER TURNING/NO SIGNAL                                                           49008
DRIVING SKILLS/KNOWLEDGE/EXPERIENCE                                                  47434
DISREGARDING TRAFFIC SIGNALS                                                         30303
WEATHER                                                                              21949

In [125]:
df.shape

(920610, 60)

In [126]:
df.head().T

,0,1,2,3,6
CRASH_RECORD_ID,79c7a2ce89f446262efd86df3d72d18b04ba487024b7c4...,79c7a2ce89f446262efd86df3d72d18b04ba487024b7c4...,792b539deaaad65ee5b4a9691d927a34d298eb33d42af0...,792b539deaaad65ee5b4a9691d927a34d298eb33d42af0...,05b1982cdba5d8a00e7e76ad1ecdab0e598429f78481d2...
RD_NO_x,JC199149,JC199149,JB422857,JB422857,JF378711
CRASH_DATE_EST_I,NaN,NaN,NaN,NaN,NaN
CRASH_DATE_x,03/25/2019 02:43:00 PM,03/25/2019 02:43:00 PM,09/05/2018 08:40:00 AM,09/05/2018 08:40:00 AM,08/29/2022 11:30:00 AM
POSTED_SPEED_LIMIT,30,30,30,30,30
TRAFFIC_CONTROL_DEVICE,TRAFFIC SIGNAL,TRAFFIC SIGNAL,NO CONTROLS,NO CONTROLS,TRAFFIC SIGNAL
DEVICE_CONDITION,FUNCTIONING PROPERLY,FUNCTIONING PROPERLY,NO CONTROLS,NO CONTROLS,FUNCTIONING PROPERLY
WEATHER_CONDITION,CLEAR,CLEAR,CLEAR,CLEAR,CLEAR
LIGHTING_CONDITION,DAYLIGHT,DAYLIGHT,DAYLIGHT,DAYLIGHT,DAYLIGHT
FIRST_CRASH_TYPE,TURNING,TURNING,ANGLE,ANGLE,REAR END


## Chopping down the data

There's just too much

Do after the initial clean to get a better sample. 

In [127]:
df.shape

(920610, 60)

In [128]:
df = df.sample(frac=0.01, random_state=42)

In [129]:
df.shape

(9206, 60)

## Cleaning Leftover Columns

### Split Into categorical and numerical

Now that they have been merged on CRASH_RECORD_ID, I'm going to remove that column, and create a new index

In [130]:
df = df.drop('CRASH_RECORD_ID', axis=1)

In [131]:
df = df.reset_index(drop=True)

In [132]:
index = df.index

In [133]:
print(index)

RangeIndex(start=0, stop=9206, step=1)


In [134]:
# split df into categorical and numerical dataframes
df_cat = df.select_dtypes(include='object')
df_num = df.select_dtypes(exclude='object')

In [135]:
# df_cat = categorical_data.reset_index(drop=True)
# df_num = numerical_data.reset_index(drop=True)

#### Numerical Cleaning

Okay so everything that is left over with the numerical columns should be able to be filled with 0.

In [136]:
# check df_num for nulls
df_num.isnull().sum()

POSTED_SPEED_LIMIT                  0
STREET_NO                           0
NUM_UNITS                           0
INJURIES_TOTAL                      5
INJURIES_FATAL                      5
INJURIES_INCAPACITATING             5
INJURIES_NON_INCAPACITATING         5
INJURIES_REPORTED_NOT_EVIDENT       5
INJURIES_NO_INDICATION              5
INJURIES_UNKNOWN                    5
CRASH_HOUR                          0
CRASH_DAY_OF_WEEK                   0
CRASH_MONTH                         0
LATITUDE                           60
LONGITUDE                          60
CRASH_UNIT_ID                       0
UNIT_NO                             0
NUM_PASSENGERS                   7603
VEHICLE_YEAR                     1418
OCCUPANT_CNT                      199
dtype: int64

In [137]:
# fill all missing values in df_num with 0
df_num.fillna(0, inplace=True)

c:\Anaconda3\envs\learn-env\lib\site-packages\pandas\core\frame.py:4317: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


#### categorical cleaning

In [138]:
df_cat.isna().sum()

RD_NO_x                      58
CRASH_DATE_EST_I           8665
CRASH_DATE_x                  0
TRAFFIC_CONTROL_DEVICE        0
DEVICE_CONDITION              0
WEATHER_CONDITION             0
LIGHTING_CONDITION            0
FIRST_CRASH_TYPE              0
TRAFFICWAY_TYPE               0
ALIGNMENT                     0
ROADWAY_SURFACE_COND          0
ROAD_DEFECT                   0
REPORT_TYPE                 281
CRASH_TYPE                    0
INTERSECTION_RELATED_I     6728
NOT_RIGHT_OF_WAY_I         8836
HIT_AND_RUN_I              6890
DAMAGE                        0
DATE_POLICE_NOTIFIED          0
PRIM_CONTRIBUTORY_CAUSE       0
SEC_CONTRIBUTORY_CAUSE        0
STREET_DIRECTION              0
STREET_NAME                   0
MOST_SEVERE_INJURY            5
LOCATION                     60
RD_NO_y                      63
CRASH_DATE_y                  0
UNIT_TYPE                    10
CMRC_VEH_I                 9018
MAKE                        199
MODEL                       199
LIC_PLAT

In [139]:
# print out the value_counts for every column in df_cat
for col in df_cat.columns:
    print(col)
    print(df_cat[col].value_counts())
    print('')

RD_NO_x
JA242863    2
JB360636    2
JF252396    2
JB422778    2
JF339257    2
           ..
JD205532    1
JE426252    1
JF500095    1
JD295019    1
JB234675    1
Name: RD_NO_x, Length: 9095, dtype: int64

CRASH_DATE_EST_I
Y    463
N     78
Name: CRASH_DATE_EST_I, dtype: int64

CRASH_DATE_x
01/20/2019 07:00:00 PM    3
08/28/2020 04:30:00 PM    2
07/27/2018 12:45:00 PM    2
01/13/2016 03:30:00 PM    2
11/05/2019 02:00:00 PM    2
                         ..
02/11/2018 03:56:00 AM    1
04/12/2018 11:40:00 AM    1
11/01/2015 12:30:00 AM    1
03/24/2022 03:23:00 PM    1
02/13/2023 04:50:00 PM    1
Name: CRASH_DATE_x, Length: 9094, dtype: int64

TRAFFIC_CONTROL_DEVICE
NO CONTROLS                 4845
TRAFFIC SIGNAL              2949
STOP SIGN/FLASHER           1044
UNKNOWN                      206
OTHER                         71
LANE USE MARKING              18
YIELD                         17
OTHER REG. SIGN               14
OTHER WARNING SIGN             9
PEDESTRIAN CROSSING SIGN       7


In [140]:
df_cat_dropped_cols = df_cat.drop(columns=['RD_NO_x',
                'CRASH_DATE_EST_I', 
                'REPORT_TYPE', 
                'STREET_DIRECTION',
                'STREET_NAME',
                'LOCATION', 
                'RD_NO_y', 
                'CMRC_VEH_I', 
                'LIC_PLATE_STATE', 
                'TRAVEL_DIRECTION'
                 ])

In [141]:
df_cat_dropped_cols.isna().sum()

CRASH_DATE_x                  0
TRAFFIC_CONTROL_DEVICE        0
DEVICE_CONDITION              0
WEATHER_CONDITION             0
LIGHTING_CONDITION            0
FIRST_CRASH_TYPE              0
TRAFFICWAY_TYPE               0
ALIGNMENT                     0
ROADWAY_SURFACE_COND          0
ROAD_DEFECT                   0
CRASH_TYPE                    0
INTERSECTION_RELATED_I     6728
NOT_RIGHT_OF_WAY_I         8836
HIT_AND_RUN_I              6890
DAMAGE                        0
DATE_POLICE_NOTIFIED          0
PRIM_CONTRIBUTORY_CAUSE       0
SEC_CONTRIBUTORY_CAUSE        0
MOST_SEVERE_INJURY            5
CRASH_DATE_y                  0
UNIT_TYPE                    10
MAKE                        199
MODEL                       199
VEHICLE_DEFECT              199
VEHICLE_TYPE                199
VEHICLE_USE                 199
MANEUVER                    199
EXCEED_SPEED_LIMIT_I       9178
FIRST_CONTACT_POINT         220
dtype: int64

## Getting target variable

In [142]:
'''
remove
PRIM_CONTRIBUTORY_CAUSE
and
SEC_CONTRIBUTORY_CAUSE
from df_cat_dropped_cols
assign it to it's own dataframe called df_targets,
and then export df_targets to csv
'''

# Specifically I'm going to make prim cause the target and remove secondary, maybe I'll come back to it if I have time.
df_targets = df_cat_dropped_cols[['PRIM_CONTRIBUTORY_CAUSE']]
df_cat_dropped_cols.drop(columns=['PRIM_CONTRIBUTORY_CAUSE', 'SEC_CONTRIBUTORY_CAUSE'], inplace=True)
df_targets.to_csv('data/df_targets.csv')

In [143]:
df_targets.isna().sum()

PRIM_CONTRIBUTORY_CAUSE    0
dtype: int64

In [144]:
df_targets.dtypes

PRIM_CONTRIBUTORY_CAUSE    object
dtype: object

In [145]:
df_targets.value_counts()

# give me the value counts for each column in df_targets
for col in df_targets.columns:
    print(col)
    print(df_targets[col].value_counts())
    print('')

PRIM_CONTRIBUTORY_CAUSE
FAILING TO YIELD RIGHT-OF-WAY                                                       1647
FOLLOWING TOO CLOSELY                                                               1508
NOT APPLICABLE                                                                       811
IMPROPER OVERTAKING/PASSING                                                          706
FAILING TO REDUCE SPEED TO AVOID CRASH                                               669
IMPROPER BACKING                                                                     596
IMPROPER LANE USAGE                                                                  553
IMPROPER TURNING/NO SIGNAL                                                           487
DRIVING SKILLS/KNOWLEDGE/EXPERIENCE                                                  465
DISREGARDING TRAFFIC SIGNALS                                                         288
WEATHER                                                                              1

## splitting into ordinal and nominal

In [146]:
'''
assign to df_cat_ordinal 
INTERSECTION_RELATED_I
NOT_RIGHT_OF_WAY_I
HIT_AND_RUN_I
and EXCEED_SPEED_LIMIT_I
'''
df_cat_ordinal = df_cat_dropped_cols[['INTERSECTION_RELATED_I',
                                        'NOT_RIGHT_OF_WAY_I',
                                        'HIT_AND_RUN_I',
                                        'EXCEED_SPEED_LIMIT_I'
                                        ]]

# assign the rest to nominal
df_cat_nominal = df_cat_dropped_cols.drop(columns=['INTERSECTION_RELATED_I',
                                        'NOT_RIGHT_OF_WAY_I',
                                        'HIT_AND_RUN_I',
                                        'EXCEED_SPEED_LIMIT_I'
                                        ])

### ordinal

In [147]:
# list nulls in ordinal
df_cat_ordinal.isna().sum()

INTERSECTION_RELATED_I    6728
NOT_RIGHT_OF_WAY_I        8836
HIT_AND_RUN_I             6890
EXCEED_SPEED_LIMIT_I      9178
dtype: int64

In [148]:
for col in df_cat_ordinal.columns:
    print(col)
    print(df_cat_ordinal[col].value_counts())
    print('')

INTERSECTION_RELATED_I
Y    2382
N      96
Name: INTERSECTION_RELATED_I, dtype: int64

NOT_RIGHT_OF_WAY_I
Y    330
N     40
Name: NOT_RIGHT_OF_WAY_I, dtype: int64

HIT_AND_RUN_I
Y    2205
N     111
Name: HIT_AND_RUN_I, dtype: int64

EXCEED_SPEED_LIMIT_I
Y    22
N     6
Name: EXCEED_SPEED_LIMIT_I, dtype: int64



In [149]:
# fill all nulls in ordinal with 'N'
df_cat_ordinal_filled = df_cat_ordinal.fillna('N')

In [150]:
df_cat_ordinal_filled.isna().sum()

INTERSECTION_RELATED_I    0
NOT_RIGHT_OF_WAY_I        0
HIT_AND_RUN_I             0
EXCEED_SPEED_LIMIT_I      0
dtype: int64

In [151]:
for col in df_cat_ordinal_filled.columns:
    print(col)
    print(df_cat_ordinal_filled[col].value_counts())
    print('')

INTERSECTION_RELATED_I
N    6824
Y    2382
Name: INTERSECTION_RELATED_I, dtype: int64

NOT_RIGHT_OF_WAY_I
N    8876
Y     330
Name: NOT_RIGHT_OF_WAY_I, dtype: int64

HIT_AND_RUN_I
N    7001
Y    2205
Name: HIT_AND_RUN_I, dtype: int64

EXCEED_SPEED_LIMIT_I
N    9184
Y      22
Name: EXCEED_SPEED_LIMIT_I, dtype: int64



### nominal

In [152]:
# list nulls in nominal
df_cat_nominal.isna().sum()

CRASH_DATE_x                0
TRAFFIC_CONTROL_DEVICE      0
DEVICE_CONDITION            0
WEATHER_CONDITION           0
LIGHTING_CONDITION          0
FIRST_CRASH_TYPE            0
TRAFFICWAY_TYPE             0
ALIGNMENT                   0
ROADWAY_SURFACE_COND        0
ROAD_DEFECT                 0
CRASH_TYPE                  0
DAMAGE                      0
DATE_POLICE_NOTIFIED        0
MOST_SEVERE_INJURY          5
CRASH_DATE_y                0
UNIT_TYPE                  10
MAKE                      199
MODEL                     199
VEHICLE_DEFECT            199
VEHICLE_TYPE              199
VEHICLE_USE               199
MANEUVER                  199
FIRST_CONTACT_POINT       220
dtype: int64

In [153]:
# assign the columnsn in df_cat_nominal with nulls to df_cat_nominal_null
df_cat_nominal_null = df_cat_nominal[['FIRST_CONTACT_POINT',
                                      'MANEUVER',
                                      'VEHICLE_USE',
                                      'VEHICLE_TYPE',
                                      'VEHICLE_DEFECT',
                                      'MODEL',
                                      'MAKE',
                                      'UNIT_TYPE',
                                      'MOST_SEVERE_INJURY']]

# assign the rest to df_cat_nominal_nonull
df_cat_nominal_nonull = df_cat_nominal.drop(columns=['FIRST_CONTACT_POINT',
                                                    'MANEUVER',
                                                    'VEHICLE_USE',
                                                    'VEHICLE_TYPE',
                                                    'VEHICLE_DEFECT',
                                                    'MODEL',
                                                    'MAKE',
                                                    'UNIT_TYPE',
                                                    'MOST_SEVERE_INJURY'])


In [154]:
# check that the split worked
df_cat_nominal_null.isna().sum()

FIRST_CONTACT_POINT    220
MANEUVER               199
VEHICLE_USE            199
VEHICLE_TYPE           199
VEHICLE_DEFECT         199
MODEL                  199
MAKE                   199
UNIT_TYPE               10
MOST_SEVERE_INJURY       5
dtype: int64

In [155]:
df_cat_nominal_nonull.isna().sum()

CRASH_DATE_x              0
TRAFFIC_CONTROL_DEVICE    0
DEVICE_CONDITION          0
WEATHER_CONDITION         0
LIGHTING_CONDITION        0
FIRST_CRASH_TYPE          0
TRAFFICWAY_TYPE           0
ALIGNMENT                 0
ROADWAY_SURFACE_COND      0
ROAD_DEFECT               0
CRASH_TYPE                0
DAMAGE                    0
DATE_POLICE_NOTIFIED      0
CRASH_DATE_y              0
dtype: int64

In [156]:
# how many columns are in df_cat_nominal_null?
print(len(df_cat_nominal_null.columns))

# how many columns are in df_cat_nominal_nonull?
print(len(df_cat_nominal_nonull.columns))

9
14


In [157]:
print(9 + 15)

24


In [158]:
# how many columsn were in df_cat_nominal?
print(len(df_cat_nominal.columns))

23


In [159]:
for col in df_cat_nominal_null.columns:
    print(col)
    print(df_cat_nominal_null[col].value_counts())
    print('')

FIRST_CONTACT_POINT
FRONT                 1852
REAR                  1260
FRONT-LEFT             598
SIDE-LEFT              584
SIDE-RIGHT             558
UNKNOWN                544
FRONT-RIGHT            531
FRONT-LEFT-CORNER      484
FRONT-RIGHT-CORNER     458
REAR-LEFT              443
OTHER                  283
REAR-LEFT-CORNER       244
REAR-RIGHT             220
TOTAL (ALL AREAS)      181
REAR-RIGHT-CORNER      173
SIDE-LEFT-REAR         128
SIDE-RIGHT-REAR        101
ROOF                    91
SIDE-LEFT-FRONT         75
NONE                    71
SIDE-RIGHT-FRONT        67
UNDER CARRIAGE          30
TOP                     10
Name: FIRST_CONTACT_POINT, dtype: int64

MANEUVER
STRAIGHT AHEAD                        4435
PARKED                                 867
SLOW/STOP IN TRAFFIC                   807
TURNING LEFT                           596
BACKING                                415
TURNING RIGHT                          343
PASSING/OVERTAKING                     283
UNKNOWN/

In [160]:
'''Fill the following columns with 'UNKNOWN' in df_cat_nominal_null:
MOST_SEVERE_INJURY
UNIT_TYPE
MAKE
MODEL
VEHICLE_DEFECT
FIRST_CONTACT_POINT
'''

df_cat_nominal_null['MOST_SEVERE_INJURY'].fillna('UNKNOWN', inplace=True)
df_cat_nominal_null['UNIT_TYPE'].fillna('UNKNOWN', inplace=True)
df_cat_nominal_null['MAKE'].fillna('UNKNOWN', inplace=True)
df_cat_nominal_null['MODEL'].fillna('UNKNOWN', inplace=True)
df_cat_nominal_null['VEHICLE_DEFECT'].fillna('UNKNOWN', inplace=True)
df_cat_nominal_null['FIRST_CONTACT_POINT'].fillna('UNKNOWN', inplace=True)

c:\Anaconda3\envs\learn-env\lib\site-packages\pandas\core\series.py:4517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


In [161]:
'''Fill the following columns with 'UNKNOWN/NA' in df_cat_nominal_null:
VEHICLE_TYPE
VEHICLE_USE
MANUEVER
'''

df_cat_nominal_null['VEHICLE_TYPE'].fillna('UNKNOWN/NA', inplace=True)
df_cat_nominal_null['VEHICLE_USE'].fillna('UNKNOWN/NA', inplace=True)
df_cat_nominal_null['MANEUVER'].fillna('UNKNOWN/NA', inplace=True)

## Combining nominal and ordinal again

In [162]:
# combine df_cat_nominal_null and df_cat_nominal_nonull into df_cat_nominal_filled
df_cat_nominal_filled = pd.concat([df_cat_nominal_null, df_cat_nominal_nonull], axis=1)

# combine df_cat_nominal_filled and df_cat_ordinal_filled into df_cat_filled
df_cat_filled = pd.concat([df_cat_nominal_filled, df_cat_ordinal_filled], axis=1)

## Combining everything again

In [163]:
# Combine df_cat_filled and df_num into df_clean

df_clean = pd.concat([df_num, df_cat_filled], axis=1)

In [164]:
df_clean.head().T

,0,1,2,3,4
POSTED_SPEED_LIMIT,35,30,25,30,30
STREET_NO,3254,3600,7331,1745,230
NUM_UNITS,2,2,4,2,2
INJURIES_TOTAL,0,0,1,2,0
INJURIES_FATAL,0,0,0,0,0
INJURIES_INCAPACITATING,0,0,0,0,0
INJURIES_NON_INCAPACITATING,0,0,0,0,0
INJURIES_REPORTED_NOT_EVIDENT,0,0,1,2,0
INJURIES_NO_INDICATION,2,2,1,0,6
INJURIES_UNKNOWN,0,0,0,0,0


# Exporting the cleaned data

In [165]:
# export df_clean to csv
df_clean.to_csv('data/df_clean.csv')

In [166]:
df_clean.shape

(9206, 47)

In [167]:
df_clean.isna().sum()

POSTED_SPEED_LIMIT               0
STREET_NO                        0
NUM_UNITS                        0
INJURIES_TOTAL                   0
INJURIES_FATAL                   0
INJURIES_INCAPACITATING          0
INJURIES_NON_INCAPACITATING      0
INJURIES_REPORTED_NOT_EVIDENT    0
INJURIES_NO_INDICATION           0
INJURIES_UNKNOWN                 0
CRASH_HOUR                       0
CRASH_DAY_OF_WEEK                0
CRASH_MONTH                      0
LATITUDE                         0
LONGITUDE                        0
CRASH_UNIT_ID                    0
UNIT_NO                          0
NUM_PASSENGERS                   0
VEHICLE_YEAR                     0
OCCUPANT_CNT                     0
FIRST_CONTACT_POINT              0
MANEUVER                         0
VEHICLE_USE                      0
VEHICLE_TYPE                     0
VEHICLE_DEFECT                   0
MODEL                            0
MAKE                             0
UNIT_TYPE                        0
MOST_SEVERE_INJURY  